# Interactively run workflow calcium imaging

+ This notebook walks you through the steps in detail to run the `workflow-calcium-imaging`.  

+ The workflow requires the calcium imaging acquired data from ScanImage or Scanbox and processed data from Suite2p or CaImAn.

+ If you haven't configured the paths, refer to [01-configure](01-configure.ipynb).

+ To overview the schema structures, refer to [02-workflow-structure](02-workflow-structure.ipynb).

+ If you need a more automatic approach to run the workflow, refer to [03-automate](03-automate-optional.ipynb).

Let's change the directory to the package root directory to load the local configuration (`dj_local_conf.json`).

In [1]:
# reload code if library changes
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import numpy as np

In [3]:
skip_ds = ['jm007/2022-06-13_a'] # corrupted metadata

## `Pipeline.py`

+ This script `activates` the DataJoint `elements` and declares other required tables.

In [4]:
import datajoint as dj
from workflow_calcium_imaging.pipeline import *
populate_settings = {'display_progress': True}

[2023-02-28 12:59:43,689][INFO]: Connecting jure@127.0.0.1:3306
[2023-02-28 12:59:43,784][INFO]: Connected jure@127.0.0.1:3306


In [5]:
# move to library
def get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init):
    all_subject_str = []
    all_session_str = [] # this will be list of lists - each nested list corresponding to one subject
    all_datetime_str = [] # for now hardcoded! (figure out how to do programmatically)

    count = 0
    for subject_str in os.listdir(root_data_dir):
        if os.path.isdir(f'{root_data_dir}/{subject_str}'):

            print(f'Subject: {subject_str}')
            all_subject_str.append(subject_str)

            all_subject_session_str = [] # sessions for this particular subject
            all_subject_datetime_str = []
            for subject_session_str in os.listdir(root_data_dir + '/' + subject_str):
                all_subject_session_str.append(subject_session_str)
                fake_session_datetime_str = fake_session_datetime_str_init[:18] + str(count) + '.000' # making fake unique time
                print('\n\n\nIMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.\n\n\n')
                print(fake_session_datetime_str)
                all_subject_datetime_str.append(fake_session_datetime_str) # here it is fake
                count += 1

            print(f'Identified sessions for subject {subject_str}: {all_subject_session_str}')

            all_session_str.append(all_subject_session_str) 
            all_datetime_str.append(all_subject_datetime_str) 
        
    return all_subject_str, all_session_str, all_datetime_str

### Clear previous metadata entries

In [6]:
# clear any previous entries
subject.Subject.delete()
Equipment.delete()

[2023-02-28 12:59:44,061][INFO]: Deleting 4 rows from `jure_imaging`.`_motion_correction__summary`
[2023-02-28 12:59:44,070][INFO]: Deleting 18 rows from `jure_scan`.`_scan_info__field`
[2023-02-28 12:59:44,949][INFO]: Deleting 470400 rows from `jure_scan`.`_scan_info__scan_file`
[2023-02-28 12:59:45,048][INFO]: Deleting 11 rows from `jure_scan`.`_scan_info`
[2023-02-28 12:59:45,095][INFO]: Deleting 1499 rows from `jure_imaging`.`__mask_classification__mask_type`
[2023-02-28 12:59:45,102][INFO]: Deleting 2 rows from `jure_imaging`.`__mask_classification`
[2023-02-28 12:59:45,151][INFO]: Deleting 2953 rows from `jure_imaging`.`__segmentation__mask`
[2023-02-28 12:59:45,163][INFO]: Deleting 2 rows from `jure_imaging`.`__segmentation`
[2023-02-28 12:59:45,183][INFO]: Deleting 2 rows from `jure_imaging`.`_motion_correction__rigid_motion_correction`
[2023-02-28 12:59:45,190][INFO]: Deleting 2 rows from `jure_imaging`.`_motion_correction`
[2023-02-28 12:59:45,199][INFO]: Deleting 2 rows from

Commit deletes? [yes, No]:  yes


[2023-02-28 12:59:47,049][INFO]: Deleting 1 rows from `jure_lab`.`equipment`


Deletes committed.


Commit deletes? [yes, No]:  yes


Deletes committed.


1

## Insert an entry into `subject.Subject`

In [7]:
subject.Subject.heading

subject              : varchar(8)                   # 
---
sex                  : enum('M','F','U')            # 
subject_birth_date   : date                         # 
subject_description="" : varchar(1024)                # 

In [8]:
from element_interface.prairieviewreader import get_pv_metadata

In [9]:
import pandas as pd

In [10]:
# custom function to populate database
root_data_dir = dj.config['custom']['imaging_root_data_dir']
fake_session_datetime_str_init = '2002-01-01 12:00:00.000' # making up session datetime (to query data use either the session/folder name or PraireView metadata)
all_subject_str, all_session_str, all_datetime_str = get_metadata_from_filetree(root_data_dir, fake_session_datetime_str_init)


Subject: jm003



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:00.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:01.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the experiment can still be accessed though through the `session` entry (YYYY-MM-DD_x) or from where the bruker metadata is stored within the database.



2002-01-01 12:00:02.000



IMPORTANT: JM made up a fake datetime to fit convention of DJ. If needed for analysis, the true datetime of the 

In [11]:
print(f'All subjects: {all_subject_str}')
print(f'All session: {all_session_str}')


All subjects: ['jm003', 'jm004', 'jm007', 'jm008', 'jm010', 'jm012', 'jm013', 'jm014']
All session: [['2022-05-03_b', '2022-05-04_a', '2022-05-04_b', '2022-05-05_b', '2022-05-05_c'], ['2022-05-06_a'], ['2022-06-13_a', '2022-06-13_b', '2022-06-14_a', '2022-06-14_b', '2022-06-15_a', '2022-06-15_b'], ['2022-06-14_a', '2022-06-14_b', '2022-06-15_a', '2022-06-15_b', '2022-06-16_a', '2022-06-16_b', '2022-06-17_a', '2022-06-17_b'], ['2022-06-15_a', '2022-06-15_c', '2022-06-16_a', '2022-06-16_b', '2022-06-17_a', '2022-06-17_b'], ['2022-07-06_a'], ['2023-02-02_a', '2023-02-02_b', '2023-03-02_a'], ['2023-03-02_a', '2023-03-02_b', '2023-03-02_c']]


In [12]:
for subject_str in all_subject_str:
    
    subject.Subject.insert1(dict(subject=subject_str, 
                                 sex='U', 
                                 subject_birth_date='2020-01-01', # FIGURE OUT HOW TO HANDLE
                                 subject_description=''))

In [13]:
subject.Subject()

subject,sex,subject_birth_date,subject_description
jm003,U,2020-01-01,
jm004,U,2020-01-01,
jm007,U,2020-01-01,
jm008,U,2020-01-01,
jm010,U,2020-01-01,
jm012,U,2020-01-01,
jm013,U,2020-01-01,
jm014,U,2020-01-01,


## Insert an entry into `lab.Equipment`

In [14]:
Equipment.insert1(dict(scanner='PrairieView'))

## Insert an entry into `session.Session`

In [15]:
session.Session.describe();

-> Subject
session_datetime     : datetime                     



In [16]:
session.Session.heading

# 
subject              : varchar(8)                   # 
session_datetime     : datetime                     # 

In [17]:
session.Session()

subject,session_datetime


In [18]:
all_datetime_str

[['2002-01-01 12:00:00.000',
  '2002-01-01 12:00:01.000',
  '2002-01-01 12:00:02.000',
  '2002-01-01 12:00:03.000',
  '2002-01-01 12:00:04.000'],
 ['2002-01-01 12:00:05.000'],
 ['2002-01-01 12:00:06.000',
  '2002-01-01 12:00:07.000',
  '2002-01-01 12:00:08.000',
  '2002-01-01 12:00:09.000',
  '2002-01-01 12:00:010.000',
  '2002-01-01 12:00:011.000'],
 ['2002-01-01 12:00:012.000',
  '2002-01-01 12:00:013.000',
  '2002-01-01 12:00:014.000',
  '2002-01-01 12:00:015.000',
  '2002-01-01 12:00:016.000',
  '2002-01-01 12:00:017.000',
  '2002-01-01 12:00:018.000',
  '2002-01-01 12:00:019.000'],
 ['2002-01-01 12:00:020.000',
  '2002-01-01 12:00:021.000',
  '2002-01-01 12:00:022.000',
  '2002-01-01 12:00:023.000',
  '2002-01-01 12:00:024.000',
  '2002-01-01 12:00:025.000'],
 ['2002-01-01 12:00:026.000'],
 ['2002-01-01 12:00:027.000',
  '2002-01-01 12:00:028.000',
  '2002-01-01 12:00:029.000'],
 ['2002-01-01 12:00:030.000',
  '2002-01-01 12:00:031.000',
  '2002-01-01 12:00:032.000']]

In [19]:
for (i, subject_str) in enumerate(all_subject_str):
    for (j, subject_session_str) in enumerate(all_session_str[i]):
        
        session_datetime_str = all_datetime_str[i][j]
        print(session_datetime_str)
        session_key = dict(subject=subject_str, session_datetime=session_datetime_str)

        session.Session.insert1(session_key)

session.Session()

2002-01-01 12:00:00.000
2002-01-01 12:00:01.000
2002-01-01 12:00:02.000
2002-01-01 12:00:03.000
2002-01-01 12:00:04.000
2002-01-01 12:00:05.000
2002-01-01 12:00:06.000
2002-01-01 12:00:07.000
2002-01-01 12:00:08.000
2002-01-01 12:00:09.000
2002-01-01 12:00:010.000
2002-01-01 12:00:011.000
2002-01-01 12:00:012.000
2002-01-01 12:00:013.000
2002-01-01 12:00:014.000
2002-01-01 12:00:015.000
2002-01-01 12:00:016.000
2002-01-01 12:00:017.000
2002-01-01 12:00:018.000
2002-01-01 12:00:019.000
2002-01-01 12:00:020.000
2002-01-01 12:00:021.000
2002-01-01 12:00:022.000
2002-01-01 12:00:023.000
2002-01-01 12:00:024.000
2002-01-01 12:00:025.000
2002-01-01 12:00:026.000
2002-01-01 12:00:027.000
2002-01-01 12:00:028.000
2002-01-01 12:00:029.000
2002-01-01 12:00:030.000
2002-01-01 12:00:031.000
2002-01-01 12:00:032.000


subject,session_datetime
jm003,2002-01-01 12:00:00
jm003,2002-01-01 12:00:01
jm003,2002-01-01 12:00:02
jm003,2002-01-01 12:00:03
jm003,2002-01-01 12:00:04
jm004,2002-01-01 12:00:05
jm007,2002-01-01 12:00:06
jm007,2002-01-01 12:00:07
jm007,2002-01-01 12:00:08
jm007,2002-01-01 12:00:09


## Insert an entry into `session.SessionDirectory`

+ The `session_dir` is the relative path to the `imaging_root_data_dir` for the given session, in POSIX format with `/`.

+ Instead of a relative path, `session_dir` could be an absolute path but it is not recommended as the absolute path would have to match the `imaging_root_data_dir` in `dj_local_conf.json`.

In [20]:
session.SessionDirectory.describe();

-> Session
---
session_dir          : varchar(256)                 # Path to the data directory for a session



In [21]:
session.SessionDirectory.heading

# 
subject              : varchar(8)                   # 
session_datetime     : datetime                     # 
---
session_dir          : varchar(256)                 # Path to the data directory for a session

In [22]:
import glob
def get_TSeries_name(subject_str, subject_session_str):
    imaging_root_data_dir = dj.config['custom']['imaging_root_data_dir']
    glob_prompt = f'{imaging_root_data_dir}/{subject_str}/{subject_session_str}/TSeries*'
    tseries_fullpaths_list = glob.glob(glob_prompt)
    
    if len(tseries_fullpaths_list) > 1: print('WARNING: more then one TSeries* folder found, only considering the FIRST ONE')
    
    tseries_name = tseries_fullpaths_list[0].split('/')[-1]
    return tseries_name

In [23]:
for (i, subject_str) in enumerate(all_subject_str):
    for (j, subject_session_str) in enumerate(all_session_str[i]):
        
        if subject_str + '/' + subject_session_str not in skip_ds:

            session_datetime_str = all_datetime_str[i][j]
            tseries_name = get_TSeries_name(subject_str, subject_session_str)

            session_dir = f'{subject_str}/{subject_session_str}/{tseries_name}'

            session.SessionDirectory.insert1(dict(subject=subject_str, 
                                                  session_datetime=session_datetime_str, 
                                                  session_dir=session_dir))

            session.SessionDirectory()

## Insert an entry into `scan.Scan`

In [24]:
scan.Scan.heading

subject              : varchar(8)                   # 
session_datetime     : datetime                     # 
scan_id              : int                          # 
---
scanner=null         : varchar(32)                  # 
acq_software         : varchar(24)                  # 
scan_notes=""        : varchar(4095)                # 

In [25]:
for (i, subject_str) in enumerate(all_subject_str):
    for (j, subject_session_str) in enumerate(all_session_str[i]):
        
        if subject_str + '/' + subject_session_str not in skip_ds:

            session_datetime_str = all_datetime_str[i][j]

            scan.Scan.insert1(dict(subject=subject_str, 
                                   session_datetime=session_datetime_str, 
                                   scan_id=0, 
                                   scanner='PrairieView', 
                                   acq_software='PrairieView',
                                   scan_notes=''))
            scan.Scan()

## Populate `scan.ScanInfo`

+ This imported table stores information about the acquired image (e.g. image dimensions, file paths, etc.).
+ `populate` automatically calls `make` for every key for which the auto-populated table is missing data.
+ `populate_settings` passes arguments to the `populate` method.
+ `display_progress=True` reports the progress bar

In [26]:
scan.ScanInfo.describe();

# General data about the reso/meso scans from header
-> scan.Scan
---
nfields              : tinyint                      # number of fields
nchannels            : tinyint                      # number of channels
ndepths              : int                          # Number of scanning depths (planes)
nframes              : int                          # number of recorded frames
nrois                : tinyint                      # number of ROIs (see scanimage's multi ROI imaging)
x=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
y=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
z=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
fps                  : float                        # (Hz) frames per second - Volumetric Scan Rate
bidirectional        : tinyint                      # true = bidirectional scannin

/home/cossart/anaconda3/envs/dj_cossart/lib/python3.8/inspect.py:350: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)
/home/cossart/anaconda3/envs/dj_cossart/lib/python3.8/inspect.py:350: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)
/home/cossart/anaconda3/envs/dj_cossart/lib/python3.8/inspect.py:350: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)


In [27]:
scan.ScanInfo.heading

# General data about the reso/meso scans from header
subject              : varchar(8)                   # 
session_datetime     : datetime                     # 
scan_id              : int                          # 
---
nfields              : tinyint                      # number of fields
nchannels            : tinyint                      # number of channels
ndepths              : int                          # Number of scanning depths (planes)
nframes              : int                          # number of recorded frames
nrois                : tinyint                      # number of ROIs (see scanimage's multi ROI imaging)
x=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
y=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
z=null               : float                        # (um) ScanImage's 0 point in the motor coordinate system
fps                  : float  

In [28]:
scan.ScanInfo.populate(**populate_settings)
scan.ScanInfo()


ScanInfo:   0%|          | 0/32 [00:00<?, ?it/s]




Scan_filepaths:
/media/cossart/DATA/dj_cossart/data/jm003/2022-05-03_b/TSeries-05032022-002/TSeries-05032022-002_Cycle00001_Ch1_014638.ome.tif






IMPORTANT: JM: I also changed default number of roi from 1 to 0, because 1 throws an error with auto-populating suite2p



{'num_fields': 1, 'num_channels': 2, 'num_planes': 1, 'num_frames': 18000, 'num_rois': 0, 'x_pos': None, 'y_pos': None, 'z_pos': None, 'frame_rate': 30.234904649390604, 'bidirectional': False, 'bidirectional_z': False, 'scan_datetime': datetime.datetime(2022, 5, 3, 8, 48, 15), 'usecs_per_line': 63.11900000000001, 'scan_duration': 604.810767535, 'height_in_pixels': 512, 'width_in_pixels': 512, 'height_in_um': 752.5374145480857, 'width_in_um': 752.5374145480857, 'fieldX': -0.02658756, 'fieldY': -0.0752376, 'fieldZ': 99.925, 'recording_time': '08:48:17.0646299'}


ScanInfo:   3%|▎         | 1/32 [00:02<01:11,  2.32s/it]




Scan_filepaths:
/media/cossart/DATA/dj_cossart/data/jm003/2022-05-04_a/TSeries-05042022-001/TSeries-05042022-001_Cycle00001_Ch2_000001.ome.tif






IMPORTANT: JM: I also changed default number of roi from 1 to 0, because 1 throws an error with auto-populating suite2p



{'num_fields': 1, 'num_channels': 1, 'num_planes': 1, 'num_frames': 18000, 'num_rois': 0, 'x_pos': None, 'y_pos': None, 'z_pos': None, 'frame_rate': 30.354921581550908, 'bidirectional': False, 'bidirectional_z': False, 'scan_datetime': datetime.datetime(2022, 5, 4, 14, 39, 18), 'usecs_per_line': 63.11, 'scan_duration': 602.502629772, 'height_in_pixels': 512, 'width_in_pixels': 512, 'height_in_um': 537.5267246772071, 'width_in_um': 537.5267246772071, 'fieldX': 0.0, 'fieldY': 0.0, 'fieldZ': 123.25, 'recording_time': '14:39:19.6892082'}


ScanInfo:   6%|▋         | 2/32 [00:03<00:51,  1.70s/it]

...

ScanInfo:  62%|██████▎   | 20/32 [01:14<00:51,  4.27s/it]




Scan_filepaths:
/media/cossart/DATA/dj_cossart/data/jm010/2022-06-15_c/TSeries-05182022-1006-003/TSeries-05182022-1006-003_Cycle00001_Ch1_000001.ome.tif






IMPORTANT: JM: I also changed default number of roi from 1 to 0, because 1 throws an error with auto-populating suite2p





/home/cossart/anaconda3/envs/dj_cossart/lib/python3.8/site-packages/datajoint/table.py:818: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if value is None or (attr.numeric and (value == "" or np.isnan(float(value)))):
ScanInfo:  62%|██████▎   | 20/32 [01:16<00:45,  3.82s/it]

z_fields.size: 1
n_depths: 2
z_step: 200.0



IMPORTANT: here I (JM) hardcoded the subindex because of the issue of two z devices (see: https://github.com/datajoint/element-interface/issues/77)






IMPORTANT: another issue can occur here, sometimes Prairie view writes the z position of planes wrongly (even though the z changes, it logs the same depth for the two planes)



Error occured in asserting "z_fields.size == n_depths", moving to next dataset (this is probably because of a Prairie view bug, sometimes it writes the z position of planes wrongly (even though the z changes, it logs the same depth for the two planes))
Overwriting n_depths to be equal to z_fields.size...
{'num_fields': 1, 'num_channels': 2, 'num_planes': 1, 'num_frames': 24000, 'num_rois': 0, 'x_pos': None, 'y_pos': None, 'z_pos': None, 'frame_rate': 30.22341267543331, 'bidirectional': False, 'bidirectional_z': True, 'scan_datetime': datetime.datetime(2022, 6, 15, 18, 56, 46), 'usecs_per_line': 63.143, 'scan_durati

DataError: (1265, "Data truncated for column 'field_z' at row 1")